## Check System Python

In [ ]:
!/Users/miki/miniconda3/bin/conda list

In [2]:
import sys
print("SYS check")
print(sys.executable)
print(sys.version)
print(sys.version_info)

SYS check
/Users/miki/miniconda3/bin/python
3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
sys.version_info(major=3, minor=11, micro=5, releaselevel='final', serial=0)


## Import packages

In [3]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from skimage import measure
from skimage.measure import regionprops, regionprops_table
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import load_img
from importlib import reload
import segmenteverygrain as seg
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from tqdm import trange
import geopandas as gpd
import fiona
%matplotlib qt
#%matplotlib notebook
##%matplotlib widget
##%matplotlib ipympl

## Load models

In [5]:
model = seg.Unet()
model.compile(optimizer=Adam(), loss=seg.weighted_crossentropy, metrics=["accuracy"])
model.load_weights('./checkpoints/seg_model');

# the SAM model checkpoints can be downloaded from: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](checkpoint="/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/sam_vit_h_4b8939.pth")

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

In [ ]:
# reload(seg)
# fname = '../images/bucegi_conglomerate_1_image.png'
# fname = '../images/A003_20201229_103823_image.png'
# fname = '../images/IMG_5208_image.png'
#fname = '/Users/miki/Dropbox/Mac/Downloads/SAM_AI/A003_20201229_103823_image.png'
#fname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/1B.006.N_002.jpg'
#fname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/volc1.jpg'
#fname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/anf16_ppl_4800.tif'

#fname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/prova.tif'
fname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/images/mag2.jpg'
# fname = '/Users/zoltan/Downloads/vecteezy_stone-pebbles-on-river-bed_3366528.jpg'

big_im = np.array(load_img(fname))
big_im_pred = seg.predict_big_image(big_im, model, I=256)
# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, grains, coords = seg.label_grains(big_im, big_im_pred, dbs_max_dist=10.0)
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, big_im, big_im_pred, coords, labels, min_area=50.0)
print("terminated!")


In [23]:
all_grains

[<POLYGON ((596 190.5, 595 190.5, 594 190.5, 593 190.5, 592 190.5, 591 190.5,...>,
 <POLYGON ((721 239.5, 720 239.5, 719 239.5, 718 239.5, 717 239.5, 716 239.5,...>,
 <POLYGON ((800 336.5, 799 336.5, 798 336.5, 797.5 336, 797.5 335, 797 334.5,...>,
 <POLYGON ((399 436.5, 398 436.5, 397 436.5, 396 436.5, 395 436.5, 394 436.5,...>,
 <POLYGON ((956 431.5, 955 431.5, 954 431.5, 953.5 431, 953 430.5, 952 430.5,...>,
 <POLYGON ((913 516.5, 912.5 516, 912 515.5, 911 515.5, 910.5 515, 910 514.5,...>,
 <POLYGON ((360 792.5, 359 792.5, 358 792.5, 357 792.5, 356 792.5, 355 792.5,...>,
 <POLYGON ((1617 986.5, 1616.5 986, 1616 985.5, 1615 985.5, 1614 985.5, 1613 ...>,
 <POLYGON ((686 1134.5, 685 1134.5, 684 1134.5, 683 1134.5, 682 1134.5, 681 1...>,
 <POLYGON ((1320 978.5, 1319.5 978, 1319 977.5, 1318.5 977, 1318 976.5, 1317....>,
 <POLYGON ((40 1064.5, 39 1064.5, 38 1064.5, 37 1064.5, 36 1064.5, 35 1064.5,...>,
 <POLYGON ((268 380.5, 267 380.5, 266 380.5, 265 380.5, 264 380.5, 263 380.5,...>,
 <PO

# Marco Filippi's implementation

In [7]:
# Filtrare e rimuovere i poligoni interrotti dal margine della figura MF

image = Image.open(fname) #mi serve per chiamare image.size[0] e image.size[1] MF

filtered_grains = []
deleted_polygons = []

margin = 10  # serve un margine (in pixel), non tutti i granuli sui bordi sono esattamente a 0 - MF

for polygon in all_grains:
    if all(
        margin < vertex[0] < image.width - margin and margin < vertex[1] < image.height - margin
        for vertex in polygon.exterior.coords
    ):
        filtered_grains.append(polygon)
    else:
        deleted_polygons.append(polygon)

        
# Creo una maschera da usare escludere i poligoni filtrati anche dalle altre liste (all_grains, labels, mask_all, grain_data, fig, ax, labels, grains, coords)
# Per ora ho applicato solo a grain_data

mask = [
    all(
        margin < vertex[0] < image.width - margin and margin < vertex[1] < image.height - margin
        for vertex in polygon.exterior.coords
    )
    for polygon in all_grains
]  

# Convert the list to a NumPy array before using ~ operator
mask = np.array(mask)

# Apply the mask to filter the DataFrame
filtered_grain_data = grain_data[mask]

# Create a DataFrame for deleted polygons
deleted_grain_data = grain_data[~mask]


# estrae uno shapefile (valutare dove inserire) - MF
data = {'geometry': filtered_grains}
gdf = gpd.GeoDataFrame(data)
output_shapefile = './poligoni.shp'
#fiona.supported_drivers  ##MZ
gdf.to_file(output_shapefile, driver='ESRI Shapefile')  ##<<=

# estrae uno shapefile (valutare dove inserire) - MF
data = {'geometry': deleted_polygons}
gdf = gpd.GeoDataFrame(data)
output_shapefile = './poligoni_deleted.shp'
gdf.to_file(output_shapefile, driver='ESRI Shapefile') ##<<=
print("terminated, shapefile saved")

terminated, shapefile saved


Use this figure to check the distribution of SAM prompts (= black dots):

In [8]:
plt.figure()
plt.imshow(big_im_pred)
plt.scatter(coords[:,0], coords[:,1], c='k');

## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)

In [9]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [ ]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [10]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 221/221 [00:01<00:00, 137.24it/s]


Plot the updated set of grains:

In [11]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [12]:
predictor = SamPredictor(sam)
predictor.set_image(big_im) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, big_im, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))

In [13]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [14]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 222/222 [00:01<00:00, 143.18it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [15]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))

Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [16]:
n_of_units = 10 # centimeters in the case of 'IMG_5208_image.png'
units_per_pixel = n_of_units/492.06 # length of scale bar in pixels

In [17]:
props = regionprops_table(labels.astype('int'), intensity_image = big_im, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [20]:
grain_data.tail(4)

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
218,219,0.051214,1077.080645,988.838710,0.291617,0.229720,-0.888165,0.822771,110.0,114.0,108.0,78.717742,76.266129,69.137097,59.0,56.0,47.0
219,220,0.073929,1082.039106,546.469274,0.349671,0.279318,0.032321,1.004231,149.0,151.0,140.0,80.446927,82.284916,78.458101,31.0,33.0,27.0
220,221,0.149924,1088.763085,1164.666667,0.553736,0.370265,-1.279775,1.557278,207.0,197.0,185.0,121.426997,117.677686,107.779614,40.0,37.0,39.0
221,222,9.937096,752.275229,715.514339,3.993311,3.267360,-1.020775,13.089322,203.0,206.0,194.0,62.861139,65.741438,64.905237,9.0,9.0,7.0


In [21]:
plt.figure()
plt.hist(grain_data['major_axis_length'], 25)
plt.xlabel('major axis length (cm)')
plt.ylabel('count');

## Save mask and grain labels to PNG files

In [22]:
dirname = '/Users/miki/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/Ricerca/SAM_AI/Segmentation/images/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Define a colormap using matplotlib
num_classes = len(all_grains)
cmap = plt.get_cmap('viridis', num_classes)
# Map each class label to a unique color using the colormap
vis_mask = cmap(labels.astype(np.uint16))[:,:,:3] * 255
vis_mask = vis_mask.astype(np.uint8)
# Save the mask as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_labels.png', vis_mask)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True